In [ ]:
import tensorflow as tf

#flag 다른 파일 사용 애매해 일단 직접 지정 , 10.5

#FLAGS = tf.app.flags.FLAGS
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_integer('batch_size', 50, 'Batch size')

#
def weight_variable(shape) :
    init = tf.fruncated_normal(shape, stddev=1)
    return tf.Variable(init)

def bias_variable(shape) :
    init = tf.constant(1,shape=shape)
    return tf.Variable(init)

def conv2d(x, W) :
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool(x, h, w) :
    return tf.nn.max_pool(x, ksize=[1,h,w,1],strides=[1,h,w,1], padding='SAME')


#이미지 불러오기



#신경망구성
# 64*64*3=12288
X = tf.placeholder(tf.float32, [None, 12288]) 

#reshape, 2,3은 이미지의 가로와 세로 길이, 4는 컬러채널의 수 RGB로 분류하려면 3
x_ = tf.reshape(X, [-1, 64, 64, 3])

# 결과물의 class 개수, spam, non spam으로 분류
Y = tf.placeholder(tf.float32, [None, 2])

# 첫번째 합성곱
w1 = weight_variable([5,5,3,32])
b1 = bias_variable([32])
L1 = tf.nn.relu(conv2d(x_,w1)+b1)
L1 = max_pool(L1,2,2)
#L1 = tf.nn.dropout(L1,keep_prob=keep_prob)

# 두번째 합성곱
w2 = weight_variable([5,5,32,64])
b2 = bias_variable([64])
L2 = tf.nn.relu(conv2d(L1,w2)+b2)
L2 = max_pool(L2,2,2)

# 세번째 합성곱
w3 = weight_variable([5,5,64,128])
b3 = bias_variable([128])
L3 = tf.nn.relu(conv2d(L2,w3)+b3)
L3 = max_pool(L3,2,2)


# full_connected, 
w_fc1 = weight_variable([16*16*128,1024])
b_fc1 = bias_variable([1024])
x_fc1 = tf.reshape(L3,[-1, 16*16*128])
L_fc1 = tf.nn.relu(tf.matmul(x_fc1, w_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
L_fc1 = tf.nn.dropout(L_fc1,keep_prob)

w_fc2 = weight_variable([1024,2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(L_fc1,w_fc2) + b_fc2 #hypothesis output

#cross entropy 정의 output
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=Y))

#train step 정의 output
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

#tensorboard 기록하기 위한 선언
cost_sum = tf.summary.scalar("cost", cross_entropy)

#정확도 계산 변수 선언 y_conv와 Y의 각 행에서 가장 큰 값이 같은지 비교
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


#saver 이용해서 모델 저장 model 폴더에 저장한다.
#SAVE_DIR = "model"
#saver = tf.train.Saver()
#checkpoint_path = os.path.join(SAVE_DIR,"model")
#ckpt = tf.train.get_checkpoint_state(SAVE_DIR)

with tf.Session() as sess:
    
    saver = tf.trainSaver(tf.global_variables())
    #checkpoint 확인하여 저장되어있으면 불러온다.
    ckpt = tf.train.get_checkpoint_state('./model')
    if ckpt and tf.train.checkpoint_exist(ckpt.model_checkpoint_path):
        saver.restore(sess,ckpt.model_checkpoint_path)
        print("restore")
    else:
        sess.run(tf.global_variables_initializer())
        print("new")
        
    
    print("학습 시작")
    
    #시작 시간 기록
    learning_start = time.time()
    learning_start_time = time.strftime("%X", time.localtime())
    
    
    for i in range(1000) :
        for b in range(ccunt):
            
#h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#h_pool1 = max_pool_2x2(h_conv1)    